##### Trial and error for dashboard 
__Larissa Voshol__  
__11/12/2023__  
__BHVF2__  

In [91]:
import numpy as np
import pandas as pd
import panel as pn
import bokeh as bk
from bokeh.plotting import figure, show
import jupyter_bokeh
pn.extension()


In [19]:
#import data
filepath_ct = '../OPC_CT_radiomics_TCIA.csv'
filepath_data = '../OPC_data_clinical.csv'

#make dataframes good
df_ct = pd.read_csv(filepath_ct)
del df_ct[df_ct.columns[0]]
df_ct.set_index("patient", inplace=True)
df_data = pd.read_csv(filepath_data)
df_data.set_index("patient", inplace=True)
df_shape

,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile
patient,,,,,,,,,,,,,,
4,0.540468,29.382158,54.364303,63.976558,61.294372,46.173586,64.876806,36047.000000,37.435671,0.604673,8727.224465,0.242107,36234,21.0
11,0.536580,20.921578,38.990604,44.721360,43.863424,33.120990,46.065171,14326.833333,26.340705,0.687436,4149.716086,0.289646,14404,14.1
12,0.479827,28.477824,59.350228,65.787537,66.850580,49.396356,68.161573,39800.833333,37.641266,0.608044,9271.372824,0.232944,39986,39.0
13,0.481861,22.561287,46.821157,52.430907,38.470768,45.221676,52.896125,18065.250000,30.897470,0.605112,5502.302457,0.304579,18150,2.0
16,0.336308,7.300546,21.707910,20.396078,23.086793,27.018512,27.018512,2032.166667,19.617722,0.644759,1203.352906,0.592153,2078,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.384389,36.016806,93.698856,86.764048,104.651804,63.505905,113.243101,84360.833333,52.907357,0.473039,19664.418897,0.233099,84614,-14.4
597,0.415840,25.254556,60.731494,62.609903,60.926185,40.804412,63.513778,22804.000000,33.599530,0.497842,7811.484381,0.342549,22916,21.0
598,0.468391,28.710600,61.296162,68.622154,69.339743,62.265560,72.166474,31999.166667,52.700886,0.414276,11765.755300,0.367689,32132,-94.5


In [3]:
basic_information = 0 
shape_features = 23
first_order_features = 38
GLCM_features = 55 
GLDM_features = 79
GLRLM_features = 93 
GLZSM_features = 109 
NGTDM_features = 125

start website

In [61]:
from bokeh.models import ColumnDataSource
color = df_data.loc[:,"Sex"]
df = pd.concat([df_shape, color], axis=1)



,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,Sex
patient,,,,,,,,,,,,,,,
4,0.540468,29.382158,54.364303,63.976558,61.294372,46.173586,64.876806,36047.000000,37.435671,0.604673,8727.224465,0.242107,36234,21.0,Male
11,0.536580,20.921578,38.990604,44.721360,43.863424,33.120990,46.065171,14326.833333,26.340705,0.687436,4149.716086,0.289646,14404,14.1,Male
12,0.479827,28.477824,59.350228,65.787537,66.850580,49.396356,68.161573,39800.833333,37.641266,0.608044,9271.372824,0.232944,39986,39.0,Male
13,0.481861,22.561287,46.821157,52.430907,38.470768,45.221676,52.896125,18065.250000,30.897470,0.605112,5502.302457,0.304579,18150,2.0,Male
16,0.336308,7.300546,21.707910,20.396078,23.086793,27.018512,27.018512,2032.166667,19.617722,0.644759,1203.352906,0.592153,2078,22.0,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.384389,36.016806,93.698856,86.764048,104.651804,63.505905,113.243101,84360.833333,52.907357,0.473039,19664.418897,0.233099,84614,-14.4,Female
597,0.415840,25.254556,60.731494,62.609903,60.926185,40.804412,63.513778,22804.000000,33.599530,0.497842,7811.484381,0.342549,22916,21.0,Male
598,0.468391,28.710600,61.296162,68.622154,69.339743,62.265560,72.166474,31999.166667,52.700886,0.414276,11765.755300,0.367689,32132,-94.5,Male


In [122]:
from bokeh.transform import factor_cmap
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

def render_plot(x, y, z, title):
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    #unique factors
    uniq = df[z].unique()

    if len(uniq) <= 2:
        color = factor_cmap(z, ['#1f77b4', '#aec7e8'], uniq)
    if len(uniq) <=20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    else:
        z=""
        color=""
        

    p = figure(title = title)
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    p.scatter(x, y, source=df, color=color, legend_field=z)
    return p

In [ ]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph
select_feature1 = pn.widgets.Select(name='Variable for x-axis', options=list(df_shape))
select_feature2 = pn.widgets.Select(name='Variable for y-axis', options=list(df_shape))
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    sidebar=[]
)


# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(
    pn.Row(
        pn.Column('# Make a graph'),
        pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features"),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title), title='Graph')
    )
)


dashboard.show();

Launching server at http://localhost:50900


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : LegendItem(id='6f2946bf-6a0d-4833-bd36-ad6974942363', ...)
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : fill_color='' [no close matches], hatch_color='' [no close matches], line_color='' [no close matches] {renderer: GlyphRenderer(id='35b1a010-3dd2-4f16-bdbb-5be147a12284', ...)}
